In [1]:
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics import accuracy_score
from scipy.stats import mode
from sklearn.ensemble import StackingClassifier







In [2]:
X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)

C:\Anaconda\envs\STAT321\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [74]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   pclass     1309 non-null   float64 
 1   name       1309 non-null   object  
 2   sex        1309 non-null   category
 3   age        1046 non-null   float64 
 4   sibsp      1309 non-null   float64 
 5   parch      1309 non-null   float64 
 6   ticket     1309 non-null   object  
 7   fare       1308 non-null   float64 
 8   cabin      295 non-null    object  
 9   embarked   1307 non-null   category
 10  boat       486 non-null    object  
 11  body       121 non-null    object  
 12  home.dest  745 non-null    object  
dtypes: category(2), float64(5), object(6)
memory usage: 115.4+ KB


In [4]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 1309 entries, 0 to 1308
Series name: survived
Non-Null Count  Dtype   
--------------  -----   
1309 non-null   category
dtypes: category(1)
memory usage: 1.5 KB


# Preprocessing

In [5]:
X.columns.tolist()

['pclass',
 'name',
 'sex',
 'age',
 'sibsp',
 'parch',
 'ticket',
 'fare',
 'cabin',
 'embarked',
 'boat',
 'body',
 'home.dest']

In [3]:
X = X.drop(['cabin','ticket','name',
 'boat',
 'body',
 'home.dest'],axis=1)

In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   pclass    1309 non-null   float64 
 1   sex       1309 non-null   category
 2   age       1046 non-null   float64 
 3   sibsp     1309 non-null   float64 
 4   parch     1309 non-null   float64 
 5   fare      1308 non-null   float64 
 6   embarked  1307 non-null   category
dtypes: category(2), float64(5)
memory usage: 54.1 KB


In [8]:
X['pclass'].isna().sum()

0

In [9]:
X['sex'].isna().sum()

0

In [10]:
X['age'].isna().sum()

263

In [11]:
X['sibsp'].isna().sum()

0

In [12]:
X['parch'].isna().sum()

0

In [13]:
X['fare'].isna().sum()

1

In [14]:
X['embarked'].isna().sum()

2

In [4]:
X['pclass'] = X['pclass'].astype('category')

In [5]:
numV =['age','fare','parch','sibsp']
catV =['pclass','sex','embarked']

In [6]:
numPipe = Pipeline(
steps =[('NumImpute',SimpleImputer(strategy="median")),('numStd',StandardScaler())])

In [7]:
catPipe = Pipeline(
steps =[('CatImpute',SimpleImputer(strategy="most_frequent"),),('Catohe',OneHotEncoder(drop='first'))]
)

In [8]:
processData = ColumnTransformer(
    transformers =[('numPipe',numPipe,numV),('catPipe',catPipe,catV)]
)

## KNN-preprocessing

In [9]:

# Convert categorical variables to integers
X["sex"] = X["sex"].map({"male": 0, "female": 1})
X["embarked"] = X["embarked"].map({"S": 0, "C": 1, "Q": 2})




# Model Selection

In [10]:
#do train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
#take a dataset; methods: logistic reg, DTC, KNN, SVM,random forest
finalPipeLog = Pipeline(
steps =[('processData',processData),('trn',LogisticRegression())])

In [12]:
modLogAcc =cross_val_score(finalPipeLog,X,y,cv=10,scoring='accuracy')
modLogAcc.mean()

0.745619495008808

In [13]:
finalPipeLog.fit(X_train,y_train).score(X_test,y_test)

0.7748091603053435

In [14]:
#Gridsearchcv
#use gridsearchcv to find the best estimator for each of the model
param_grid_log = {
    'trn__penalty': ['l1', 'l2'],
    'trn__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}


In [15]:
grid_log = GridSearchCV(finalPipeLog, param_grid=param_grid_log, cv=10)

In [16]:
grid_log.fit(X_train, y_train)

C:\Anaconda\envs\STAT321\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
70 fits failed out of a total of 140.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
70 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda\envs\STAT321\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda\envs\STAT321\lib\site-packages\sklearn\pipeline.py", line 406, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Anaconda\envs\STAT321\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty,

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('processData',
                                        ColumnTransformer(transformers=[('numPipe',
                                                                         Pipeline(steps=[('NumImpute',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('numStd',
                                                                                          StandardScaler())]),
                                                                         ['age',
                                                                          'fare',
                                                                          'parch',
                                                                          'sibsp']),
                                                                        ('catPipe',
                                                                         Pipeline(steps=[('CatImpute',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('Catohe',
                                                                                          OneHotEncoder(drop='first'))]),
                                                                         ['pclass',
                                                                          'sex',
                                                                          'embarked'])])),
                                       ('trn', LogisticRegression())]),
             param_grid={'trn__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'trn__penalty': ['l1', 'l2']})

In [17]:
bestmodellog = grid_log.best_estimator_

In [18]:
print("Logistic Regression best estimator accuracy on testing data:",grid_log.best_estimator_.score(X_test, y_test))
print("Logistic Regression best estimator mean cv score:",grid_log.best_score_)



Logistic Regression best estimator accuracy on testing data: 0.7786259541984732
Logistic Regression best estimator mean cv score: 0.7955402930402931


In [ ]:
#Using gridsearch cv best estimator, mean cv scores improves to 0.7955 from intial model cv score of 0.745

In [21]:
#DTC
finalPipeTree = Pipeline(
steps =[('processData',processData),('decMod',DecisionTreeClassifier())])

In [22]:
modLogdt =cross_val_score(finalPipeTree,X,y,cv=10,scoring='accuracy')
modLogdt.mean()

0.7219201409277745

In [23]:
finalPipeTree.fit(X_train,y_train).score(X_test,y_test)

0.7442748091603053

In [24]:
#gridsearchcv- DTC
param_grid_dt = {
    'decMod__max_depth': [3, 5, 7, 10],
    'decMod__min_samples_split': [2, 5, 10]
}

In [25]:
grid_dt = GridSearchCV(finalPipeTree, param_grid=param_grid_dt, cv=10)


In [26]:
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('processData',
                                        ColumnTransformer(transformers=[('numPipe',
                                                                         Pipeline(steps=[('NumImpute',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('numStd',
                                                                                          StandardScaler())]),
                                                                         ['age',
                                                                          'fare',
                                                                          'parch',
                                                                          'sibsp']),
                                                                        ('catPipe',
                                                                         Pipeline(steps=[('CatImpute',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('Catohe',
                                                                                          OneHotEncoder(drop='first'))]),
                                                                         ['pclass',
                                                                          'sex',
                                                                          'embarked'])])),
                                       ('decMod', DecisionTreeClassifier())]),
             param_grid={'decMod__max_depth': [3, 5, 7, 10],
                         'decMod__min_samples_split': [2, 5, 10]})

In [27]:
print("Best estimator for Decision Tree:", grid_dt.best_estimator_)

Best estimator for Decision Tree: Pipeline(steps=[('processData',
                 ColumnTransformer(transformers=[('numPipe',
                                                  Pipeline(steps=[('NumImpute',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('numStd',
                                                                   StandardScaler())]),
                                                  ['age', 'fare', 'parch',
                                                   'sibsp']),
                                                 ('catPipe',
                                                  Pipeline(steps=[('CatImpute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('Catohe',
                                                              

In [28]:
bestmodeldtc = grid_dt.best_estimator_

In [29]:
# Evaluate the best estimator on the testing data
print("Decision Tree best estimator accuracy on testing data:", grid_dt.best_estimator_.score(X_test, y_test))
print("Decision Tree best estimator mean cv score:", grid_dt.best_score_)


Decision Tree best estimator accuracy on testing data: 0.7786259541984732
Decision Tree best estimator mean cv score: 0.824102564102564


In [ ]:
#Similarly, dtc best estimator using gridsearch cv shows a higher mean cv score than the intial dt hyperparameters

In [30]:
#RFC
rndf = RandomForestClassifier()
finalRFC = Pipeline(steps = [('processData',processData),('RFC',rndf)])


In [32]:
finalRFC.fit(X_train,y_train).score(X_test,y_test)

0.8129770992366412

In [33]:
modrndf =cross_val_score(finalRFC,X,y,cv=10,scoring='accuracy')
modrndf.mean()

0.7539812096300647

In [34]:
bestmodelrfc=finalRFC

In [35]:
finalPipeSVM = Pipeline(steps =[('processData', processData),('SVM', SVC())])

In [36]:
finalPipeSVM.fit(X_train,y_train).score(X_test,y_test)

0.7748091603053435

In [37]:
modsvm=cross_val_score(finalPipeSVM,X,y,cv=10,scoring='accuracy')
modsvm.mean()

0.7463711098062242

In [46]:
#GridSearchCV is taking too much time to complete,
#we can try using RandomizedSearchCV instead. RandomizedSearchCV randomly samples a subset of the hyperparameter space and evaluates the performance of each sampled combination. 
#This approach can be much faster than GridSearchCV, especially when the hyperparameter space is large.

In [38]:
param_distributions_svm = {
    'SVM__C': uniform(loc=0, scale=100),
    'SVM__kernel': ['linear', 'rbf', 'sigmoid'],
    'SVM__gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
    'SVM__degree': [2, 3, 4, 5],
    'SVM__coef0': uniform(loc=0, scale=1)
}

In [39]:
random_search_svm = RandomizedSearchCV(finalPipeSVM, param_distributions=param_distributions_svm, cv=10, n_iter=100)


In [40]:
random_search_svm.fit(X_train, y_train)


RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('processData',
                                              ColumnTransformer(transformers=[('numPipe',
                                                                               Pipeline(steps=[('NumImpute',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('numStd',
                                                                                                StandardScaler())]),
                                                                               ['age',
                                                                                'fare',
                                                                                'parch',
                                                                                'sibsp']),
                                                                              ('catPipe',
                                                                               Pipeline(steps=[('CatImpute',
                                                                                                SimpleImputer(strategy='most_frequent')),
                                                                                               ('Catohe',
                                                                                                OneHotEncoder(drop='first...
                                                                                'embarked'])])),
                                             ('SVM', SVC())]),
                   n_iter=100,
                   param_distributions={'SVM__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025DD49D9400>,
                                        'SVM__coef0': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025DD51B78B0>,
                                        'SVM__degree': [2, 3, 4, 5],
                                        'SVM__gamma': ['scale', 'auto', 0.1,
                                                       0.01, 0.001],
                                        'SVM__kernel': ['linear', 'rbf',
                                                        'sigmoid']})

In [41]:
print("Best estimator for SVM:", random_search_svm.best_estimator_)


Best estimator for SVM: Pipeline(steps=[('processData',
                 ColumnTransformer(transformers=[('numPipe',
                                                  Pipeline(steps=[('NumImpute',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('numStd',
                                                                   StandardScaler())]),
                                                  ['age', 'fare', 'parch',
                                                   'sibsp']),
                                                 ('catPipe',
                                                  Pipeline(steps=[('CatImpute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('Catohe',
                                                                   OneHo

In [42]:
print("SVM best estimator score:",random_search_svm.best_estimator_.score(X_test, y_test))
print("SVM best estimator mean cv score:",random_search_svm.best_score_)

#Improved hyperparameters of svm model built using randomsearch helped in increasing the accuracy to 0.82 compared to the initial model

SVM best estimator score: 0.7862595419847328
SVM best estimator mean cv score: 0.8183974358974359


In [43]:
bestmodelsvm=random_search_svm.best_estimator_

In [44]:
#KNN estimator
finalPipekNN = Pipeline(
    steps =[('processData',processData),('KNN',KNeighborsClassifier())]
)


In [45]:
finalPipekNN.fit(X_train,y_train).score(X_test,y_test)

0.7519083969465649

In [46]:
modkNNAcc =cross_val_score(finalPipekNN,X,y,cv=10,scoring='accuracy')
modkNNAcc.mean()

0.7272753963593658

In [47]:
paramskNN = {
    'KNN__n_neighbors': [3, 5, 7, 9, 11],
    'KNN__weights': ['uniform', 'distance'],
    'KNN__p': [1, 2, 3]
}


In [48]:
grid_searchkNN = GridSearchCV(finalPipekNN, param_grid=paramskNN, cv=10, n_jobs=-1)


In [49]:
grid_searchkNN.fit(X_train,y_train)



GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('processData',
                                        ColumnTransformer(transformers=[('numPipe',
                                                                         Pipeline(steps=[('NumImpute',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('numStd',
                                                                                          StandardScaler())]),
                                                                         ['age',
                                                                          'fare',
                                                                          'parch',
                                                                          'sibsp']),
                                                                        ('catPipe',
                                                                         Pipeline(steps=[('CatImpute',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('Catohe',
                                                                                          OneHotEncoder(drop='first'))]),
                                                                         ['pclass',
                                                                          'sex',
                                                                          'embarked'])])),
                                       ('KNN', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'KNN__n_neighbors': [3, 5, 7, 9, 11],
                         'KNN__p': [1, 2, 3],
                         'KNN__weights': ['uniform', 'distance']})

In [50]:
print("Best estimator for KNN:",grid_searchkNN.best_estimator_)


Best estimator for KNN: Pipeline(steps=[('processData',
                 ColumnTransformer(transformers=[('numPipe',
                                                  Pipeline(steps=[('NumImpute',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('numStd',
                                                                   StandardScaler())]),
                                                  ['age', 'fare', 'parch',
                                                   'sibsp']),
                                                 ('catPipe',
                                                  Pipeline(steps=[('CatImpute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('Catohe',
                                                                   OneHo

In [51]:
print("KNN best estimator score:",grid_searchkNN.best_estimator_.score(X_test, y_test))
print("KNN best estimator score:",grid_searchkNN.best_score_)


KNN best estimator score: 0.7633587786259542
KNN best estimator score: 0.8012820512820513


In [53]:
bestmodelknn = grid_searchkNN.best_estimator_

# Voting Classifier

In [54]:
# voting classifier- manual

from sklearn.ensemble import VotingClassifier

estimatorsVC = [('log',bestmodellog), ('Dtc',bestmodeldtc), ('kNN',bestmodelknn), ('RFC',bestmodelrfc), ('SVM',bestmodelsvm)]
hardvc = VotingClassifier(estimators = estimatorsVC, voting = 'hard')
hardvc.fit(X_train, y_train)
y_pred_hardvc = hardvc.predict(X_test)
hard_acc = accuracy_score(y_test,y_pred_hardvc)
print("hard voting classifer accuracy score:",hard_acc)

bestmodellog.fit(X_train, y_train)
bestmodeldtc.fit(X_train, y_train)
bestmodelknn.fit(X_train, y_train)
bestmodelrfc.fit(X_train, y_train)
bestmodelsvm.fit(X_train, y_train)




logvote = sum(y_pred_hardvc == bestmodellog.predict(X_test))
dtcvote = sum(y_pred_hardvc == bestmodeldtc.predict(X_test))
knnvote = sum(y_pred_hardvc == bestmodelknn.predict(X_test))
rfcvote = sum(y_pred_hardvc == bestmodelrfc.predict(X_test))
svmvote = sum(y_pred_hardvc == bestmodelsvm.predict(X_test))



## Note: Highest vote count will determine the final prediction.
print('Logistic Regression votes:', logvote)
print('Decision Tree votes:', dtcvote)
print('KNN votes:', knnvote)
print('Random Forest votes:', rfcvote)
print('SVM votes:', svmvote)



hard voting classifer accuracy score: 0.7900763358778626
Logistic Regression votes: 253
Decision Tree votes: 257
KNN votes: 235
Random Forest votes: 228
SVM votes: 259


In [55]:
#Stacking classifier- using stackingclassifier function
estimators = [('log',bestmodellog), ('Dtc',bestmodeldtc), ('kNN',bestmodelknn), ('RFC',bestmodelrfc), ('SVM',bestmodelsvm)]

finalSTCl = StackingClassifier(estimators = estimators, final_estimator = LogisticRegression())
accSTCl =finalSTCl.fit(X_train,y_train).score(X_test,y_test)
print("STC with logit accuracy:",accSTCl)

finalSTCdt = StackingClassifier(estimators = estimators, final_estimator =DecisionTreeClassifier())
accSTCdt =finalSTCdt.fit(X_train,y_train).score(X_test,y_test)
print("STC with decision tree accuracy:",accSTCdt)

finalSTCkNN = StackingClassifier(estimators = estimators, final_estimator =KNeighborsClassifier())
accSTCkNN =finalSTCkNN.fit(X_train,y_train).score(X_test,y_test)
print("STC with KNN accuracy:",accSTCkNN)


STC with logit accuracy: 0.7709923664122137
STC with decision tree accuracy: 0.7290076335877863
STC with KNN accuracy: 0.7595419847328244


## Stacking classifier-manual

In [57]:

bestmodellog.fit(X_train, y_train)
bestmodeldtc.fit(X_train, y_train)
bestmodelknn.fit(X_train, y_train)
bestmodelrfc.fit(X_train, y_train)
bestmodelsvm.fit(X_train, y_train)

# Make predictions on test data using base models
log_preds = bestmodellog.predict(X_test)
dtc_preds = bestmodeldtc.predict(X_test)
knn_preds = bestmodelknn.predict(X_test)
rfc_preds = bestmodelrfc.predict(X_test)
svm_preds = bestmodelsvm.predict(X_test)

# Create a matrix of predicted class labels for each base model
pred_matrix = np.column_stack((log_preds, dtc_preds, knn_preds, rfc_preds, svm_preds))

log_reg = LogisticRegression()
log_reg.fit(pred_matrix, y_test)


# Calculate accuracy of the ensemble model on the test data
ensemble_preds = log_reg.predict(pred_matrix)
acc_ensemble = np.mean(ensemble_preds == y_test)
print("Ensemble model accuracy with logit:", acc_ensemble)
a = accuracy_score(ensemble_preds,y_test)
print("manual STC with logit:",a)





Ensemble model accuracy with logit: 0.8206106870229007
manual STC with logit: 0.8206106870229007


In [63]:
STCdtc = DecisionTreeClassifier()
STCdtc.fit(pred_matrix, y_test)


ensemble_preds_dtc = STCdtc.predict(pred_matrix)
adtc = accuracy_score(ensemble_preds_dtc,y_test)
print("manual STC with DTC:",adtc)

manual STC with DTC: 0.8206106870229007


In [74]:
STCKNN = KNeighborsClassifier()

STCKNN.fit(pred_matrix, y_test)

ensemble_preds_kNN = STCKNN.predict(pred_matrix)
akNN = accuracy_score(ensemble_preds_kNN,y_test)
print("manual STC with kNN:",akNN)


manual STC with kNN: 0.8091603053435115


## Stacking classifier manual- alternate way

In [67]:

# Predictions are made and combined using the base model

train_predictions = [] 

# Variable "train_predictions" is created - It's an empty list that will store predictions of base models for testing set
test_predictions = []

# A loop is created for all 5 base models - Logistic Regression, Decision Tree Classifier, KNN, Random Forest and SVM 
for model in [bestmodellog, bestmodeldtc,bestmodelknn,bestmodelrfc, bestmodelsvm]:
    model.fit(X_train, y_train) # Each model is fit on training data
    
    train_predictions.append(model.predict(X_train)) # Predictions are made for training data and stored in variable 'train_predictions' list
    test_predictions.append(model.predict(X_test)) # Predictions are made for testing data and stored in variable 'test_predictions' list
    
    
# All predictions of base models for training data are compiled horizontally in a list
train_stacked = np.column_stack(train_predictions)

# All predictions of base models for testing data are compiled horizontally in a list
test_stacked = np.column_stack(test_predictions)

# Meta models


mmodelknn =  KNeighborsClassifier()



# Meta model fit on stacked features and then predictions are made using meta model for testing data 
mmodell = LogisticRegression()
mmodell.fit(train_stacked, y_train)
y_predl = mmodell.predict(test_stacked)


# Accuracy score of stacking classifier
stacking_accl = accuracy_score(y_test, y_predl)
print(stacking_accl)









0.7900763358778626


In [71]:
mmodeldtc = DecisionTreeClassifier()
mmodeldtc.fit(train_stacked, y_train)
y_preddtc = mmodeldtc.predict(test_stacked)

stacking_accdtc = accuracy_score(y_test, y_preddtc)
print(stacking_accdtc)


0.7977099236641222


In [72]:
mmodelknn.fit(train_stacked, y_train)
y_predknn = mmodelknn.predict(test_stacked)

stacking_accknn = accuracy_score(y_test, y_predknn)
print(stacking_accknn)

0.7938931297709924
